In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
%matplotlib inline

In [2]:
df = pd.read_csv('chefkoch.csv', names = ['Zutaten', 'Bewertung', 'Gericht', 'Bewertungen_Counter', 'URL'])
#df

In [3]:
df.dtypes

Zutaten                object
Bewertung              object
Gericht                object
Bewertungen_Counter    object
URL                    object
dtype: object

In [4]:
def clean_columns(row):
    if row['URL'] is np.nan:
        #print(row['Bewertung'])
        row['URL'] = row['Bewertungen_Counter']
        row['Bewertungen_Counter'] = 0
        row['Gericht'] = row['Bewertung']
        row['Bewertung'] = np.nan
    return row

In [5]:
df.apply(clean_columns, axis = 1)
df = df.drop_duplicates(subset = 'URL')
#df

In [6]:
df.Bewertungen_Counter = df.Bewertungen_Counter.str.replace('(', '')
df.Bewertungen_Counter = df.Bewertungen_Counter.str.replace(')', '').astype(float)
df.Bewertung = df.Bewertung.str.replace(',','.')
df.Bewertung = df.Bewertung.astype(float)

In [7]:
#df.sort_values('Bewertung', ascending = False)

In [8]:
df.dtypes

Zutaten                 object
Bewertung              float64
Gericht                 object
Bewertungen_Counter    float64
URL                     object
dtype: object

In [9]:
df.Zutaten.str.extract(r'(Ei)').count().tolist()[0]

25014

In [10]:
def quickcheck_ingredients(list_):    
    return_list=[]
    for ing in list_:
        temp_dict={}
        regx = "\'("+ing+")\'"
        #regx = "'"+ing+"'"
        print(regx)
        hits = df.Zutaten.str.extract(regx).count().tolist()[0]
        print(hits)
        #print(df.Zutaten.str.extract(regx))
        temp_dict['Treffer'] = hits
        temp_dict['Zutat'] = ing
        return_list.append(temp_dict)
    return return_list

In [11]:
df_temp = pd.DataFrame(quickcheck_ingredients(['Milch', 'Ei', 'Zucker', 'Salz', 'Butter']))

'(Milch)'
8693
'(Ei)'
0
'(Zucker)'
21242
'(Salz)'
21517
'(Butter)'
13075


In [12]:
def get_Zutaten(row):
    diese_zutaten = []
    for zutat in eval(row['Zutaten']):
        for z in zutat:
            clean_z = z.split('(')[0]
            clean_z = clean_z.split(',')[0]
            #clean_z = clean_z.split(',')[0]
            clean_z = clean_z.strip()
            if ':' not in clean_z:
                if ' oder' in clean_z:
                        or_split = clean_z.split(' oder')
                        for n in or_split:
                            diese_zutaten.append(n.strip())
                elif 'und/oder' in clean_z:
                    or_split = clean_z.split(' und/oder')
                    for n in or_split:
                            diese_zutaten.append(n.strip())
                else:
                    diese_zutaten.append(clean_z)
    return diese_zutaten

In [13]:
all_zutaten = []
all_zutaten = all_zutaten + df.apply(get_Zutaten, axis = 1)
#all_zutaten

In [14]:
for zut in all_zutaten:
    for z in zut:
        if ':' in z:
            print(z)

In [15]:
long_list = []
for sublist in all_zutaten:
    for val in sublist:
        long_list.append(val)

In [16]:
len(long_list)

698672

In [17]:
long_list.count('Mehl')

19672

In [18]:
long_string = (' ').join(long_list)
len(long_string)

6751143

In [75]:
#top01 = top02 = top03 = top04 = top05 = top06 = top07 = top08 = top09 = top10 = {'name':'', 'anzahl':100}
def get_count(list_):
    set_ = set(list_)
    dict_ = {}
    anzahl_liste = []
    #top01 = top02 = top03 = top04 = top05 = top06 = top07 = top08 = top09 = top10 = {'name':'', 'anzahl':10}
    for thing in set_:        
        #dict_ = {}
        dict_[thing] = list_.count(thing)
        #anzahl_liste.append(dict_)
    return dict_

In [76]:
%%time
anzahl_liste = get_count(long_list)

CPU times: user 1min 46s, sys: 39.4 ms, total: 1min 46s
Wall time: 1min 46s


In [67]:
top01

{'name': 'Zucker', 'anzahl': 28998}

In [68]:
top02

{'name': 'Zucker', 'anzahl': 28998}

In [69]:
top03

{'name': 'Zucker', 'anzahl': 28998}

In [80]:
anzahl_liste

{'': 233,
 'Butter und 1 TL Dattelpaste': 1,
 'Naturjoghurt 1 % Fett': 1,
 'Stärke': 3,
 'Crème fraîche o. Cremefine z. Verfeinern': 2,
 'Kuchenglasur': 519,
 'Salatkräuter': 51,
 'Milch zum Einweichen des Brötchens': 1,
 'Cocktailtomaten zum Garnieren': 1,
 'Rote Bete - Blätter': 1,
 'Leinsamenmehl': 9,
 'Pflanzencreme mit Butteraroma': 1,
 'Lachsschinken': 5,
 'Kräuter zum Garnieren': 9,
 'Bohnen in Tomatensauce': 1,
 'Cashewmus': 6,
 'Marshmallow Creme': 4,
 'Peccorino': 2,
 'Wurst in Streifen geschnitten': 1,
 'Hartweizenmehl': 4,
 'Gänsebrust von etwa 1 kg': 1,
 'Aprikosensaft von den Früchten aus der Dose': 1,
 'Brennnesseln -Samen': 1,
 'Majoranblättchen': 1,
 'Kakaopulver und Milchschnitten': 1,
 'Wellfleisch': 1,
 'Fett und Mehl für die Form': 69,
 'Obstler': 3,
 'Käse gerieben': 4,
 'Geflügefond': 1,
 'Knoblauch - Butter / Kräuterbutter': 1,
 'Haferflocken für die Form': 4,
 'Fisch Heringe': 1,
 'Konfitüre zum Füllen': 1,
 'Gänseschmalz': 40,
 'Kichererbsennudeln': 1,
 'Oblat

In [94]:
%%time 
import collections
# dictionary sorted by value
order_anzahl_liste = collections.OrderedDict(sorted(anzahl_liste.items(), key=lambda t: t[1]))

CPU times: user 15.4 ms, sys: 0 ns, total: 15.4 ms
Wall time: 15.2 ms


In [102]:
order_anzahl_liste.items()

odict_items([('Butter und 1 TL Dattelpaste', 1), ('Naturjoghurt 1 % Fett', 1), ('Milch zum Einweichen des Brötchens', 1), ('Cocktailtomaten zum Garnieren', 1), ('Rote Bete - Blätter', 1), ('Pflanzencreme mit Butteraroma', 1), ('Bohnen in Tomatensauce', 1), ('Wurst in Streifen geschnitten', 1), ('Gänsebrust von etwa 1 kg', 1), ('Aprikosensaft von den Früchten aus der Dose', 1), ('Brennnesseln -Samen', 1), ('Majoranblättchen', 1), ('Kakaopulver und Milchschnitten', 1), ('Wellfleisch', 1), ('Geflügefond', 1), ('Knoblauch - Butter / Kräuterbutter', 1), ('Fisch Heringe', 1), ('Konfitüre zum Füllen', 1), ('Kichererbsennudeln', 1), ('Oblatenlebkuchen', 1), ('Rucola von der o. g. Menge abgezweigt', 1), ('Bratensaft bzw. Bratenfond', 1), ('Toastbrot - Scheibe', 1), ('Milch und Ei', 1), ('japanischer/koreanischer Reis', 1), ('Mark von der Passionsfrucht', 1), ('Hähnchen ca. 2 kg', 1), ('Öl aus dem Tomatenglas', 1), ('10 - 20 g Zartbitterschokolade mind. 75 % Kakao', 1), ('getrocknteter', 1), ('K